### Our Recommendation System
We will be design a more robust recommendation process, carried out with graph data.

Recommendation systems are used heavily in customer-facing applications, to show the user a product that they may be interested in. Product recommendations are often based on what behaviorally similar users have played and purchased.

##### Generic MySQL to igraph methods
We will write a set of reusable, geneeric methods to create an igraph graph from columns in a MySQL table. The functions will be designed to create a heterogeneous, bipartile, directed graph, give a set of column names.

In [2]:
import igraph as ig
from graphtastic.database.mysql import query_mysql
from graphtastic.igraph.helper import create_igraph_ids

In [1]:
def mysql_to_graph(table, source, target, weights, password):
    
    sql_query = f"SELECT {source}, {target}, {weights} FROM {table};"
    
    data = query_mysql(sql_query, password = password)
    
    source_nodes = sorted(list(set([source for source, _, _ in data])))
    
    target_nodes = sorted(list(set([target for _, target, _ in data])))
    
    # Create a dictionaries using list comprehension
    source_igraph_ids = create_igraph_ids(source_nodes)
    target_igraph_ids = create_igraph_ids(target_nodes, len(source_igraph_ids))
    
    # we need to generate an igraph edge list using the pairs in the created dictionaries
    edges = [(source_igraph_ids[source], target_igraph_ids[target]) for source, target, _ in data]
    weights = [weight for _, _, weight in data]
    
    # The data is ready to be added to an igraph graph.
    g = ig.Graph(directed=True)
    g.add_vertices(len(source_nodes + target_nodes))
    g.vs['internal_id'] = list(source_igraph_ids.keys()) + list(target_igraph_ids.keys())
    g.vs['type'] = ['source' for _ in source_nodes] + ['target' for _ in target_nodes]
    g.add_edges(edges)
    g.es['weight'] = weights
    
    return g

In [4]:
g = mysql_to_graph('steam_play', 'id', 'game_name', 'hours', password='')

#### A more advanced recommendation system using Jaccard similarity
To recommend games more smartly, we are going to implement a solution based on node similarity. Node similarity compares the neighbouring nodes of two nodes and return a score. In our method, we will use Jaccard similarity, available in igraph as `similarity_jaccard()`. The score resulting from `similarity_jaccard()` will be equal to the count of two nodes common neighbors, divided by the number of neighborrs adjacent to atleast one of the two nodes.

Relating to our use case, in our Steam graph, user nodes are adjacent to game nodes. So, user nodes that
share a high Jaccard similarity with other user nodes play similar games. We can use this information to
make new recommendations to our users.

In [13]:
def prune_graph(g, min_hours):
    edges_to_remove = g.es.select(weight_lt=min_hours)
    g.delete_edges(edges_to_remove)
    return g

In [14]:
## Let's define a new function, make_recommendations()
def make_recommendation(g, user, min_hours):
    
    # Lets find the node represented by our user parameter. This wil return a list of one element.
    user_node = g.vs.select(internal_id_eq=user)
    user_node = user_node[0].index
    
    # we will use this function to remove edges in our graph based on the number of hours that games have been played
    g = prune_graph(g, min_hours)
    
    other_user_nodes = g.vs.select(type_eq='source')
    
    pairs = [[user_node, other_user.index] for other_user in other_user_nodes if other_user.index != user_node]
    similarities = g.similarity_jaccard(pairs=pairs, mode='out')
    node_similarity = [[pair[1], similarity] for pair, similarity in zip(pairs, similarities)]
    node_similarity = sorted(node_similarity, key=lambda x: x[1], reverse=True)
    most_similar_node = node_similarity[0][0]
    game_recommendations = g.vs[g.neighbors(most_similar_node)]['internal_id']
    owned_games = g.vs[g.neighbors(user_node)]['internal_id']
    new_games = [game for game in game_recommendations if game not in owned_games]
    return new_games

In [17]:
g = mysql_to_graph('steam_play', 'id', 'game_name', 'hours', password='')
recommendations = make_recommendation(g, '87907200', min_hours=0.5)
print(recommendations)

['BLOCKADE 3D', 'Counter-Strike Global Offensive', 'Guns of Icarus Online']
